In [95]:
import numpy as np
import pandas as pd
import os
import re

# Load Data + Calculate Payoffs

In [123]:
def load_csv(filename, skip_rows):
    df = pd.read_csv(filename, skiprows=skip_rows, header=0)
    df.drop(df.columns[0], axis=1, inplace=True)
    return df

def calculate_avg(df): # prints dataframe to check
    df_even = df.iloc[:, 0::6]
    display(df_even)
    even_row_avg = df_even.iloc[-1].mean()
    print('AVG Cumulative Profit of retailer 0:', even_row_avg)
    
    df_odd = df.iloc[:, 3::6]
    display(df_odd)
    odd_row_avg = df_odd.iloc[-1].mean()
    print('AVG Cumulative Profit of retailer 1:', odd_row_avg)

def calculate_payoffs(df):
    retailer_0 = df.iloc[:, 0::6]
    retailer_0_payoff = round(retailer_0.iloc[-1].mean(), 2)
    
    retailer_1 = df.iloc[:, 3::6]
    retailer_1_payoff = round(retailer_1.iloc[-1].mean(), 2)
    return (retailer_0_payoff, retailer_1_payoff)

def get_column_dominant(lst):
    payoffs = [eval(s)[1] for s in lst]
    max_payoff = max(payoffs)
    dominant_index = payoffs.index(max_payoff)
    opponent_payoff = lst[dominant_index].strip('[').split(',')[0]
    dominant_str = f'[{opponent_payoff}, {max_payoff}*]'
    return (dominant_index, dominant_str)

def get_row_dominant(lst):
    payoffs = [float(s.strip('[').split(',')[0]) for s in lst]
    max_payoff = max(payoffs)
    dominant_index = payoffs.index(max_payoff)
    opponent_payoff = lst[dominant_index].strip(']').split(', ')[1]
    dominnant_str = f'[{max_payoff}*, {opponent_payoff}]'
    return (dominant_index, dominnant_str)

def construct_payoff_table():
    columns = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']
    payoffs = [['x' for y in range(6)] for x in range(6)]
    
    # generate payoff table
    for i in range(1, len(payoffs)+1):
        for j in range(1, len(payoffs)+1):
            filename = f'./experiment_data/S{i}_S{j}.csv'
            if os.path.isfile(filename):
                df = load_csv(filename, 34)
                payoff =  calculate_payoffs(df)
                payoff_str = f'[{payoff[0]}, {payoff[1]}]'
                payoffs[i-1][j-1] = payoff_str

    # mark dominant strategies in retailer 1's perspective
    for i in range(0, len(payoffs)):
        cur_row = payoffs[i]
        (dominant_index, dominant_str) = get_column_dominant(cur_row)
        payoffs[i][dominant_index] = dominant_str
    
    # mark dominant strategies in retailer 0's perspective
    for i in range(0, len(payoffs)):
        cur_column = [row[i] for row in payoffs]
        (dominant_index, dominant_str) = get_row_dominant(cur_column)
        payoffs[dominant_index][i] = dominant_str

    # convert to dataframe
    table = pd.DataFrame(payoffs, columns=columns)
    table = table.rename(index={0: 'S1', 1: 'S2', 2: 'S3', 3: 'S4', 4: 'S5', 5: 'S6'})
    display(table)

def cal_avg(po1, po2):
    avg_1 = round((po1[0] + po2[0])/2, 2)
    avg_2 = round((po1[1] + po2[1])/2, 2)
    return (avg_1, avg_2)

def construct_payoff_table_for_weightings(columns, num_row, num_column, naming, weighting, index_dict, row_start=0, column_start=0):
    weighting_str = f'{naming}{weighting}'
    payoffs = [['x' for y in range(num_column)] for x in range(num_row)]
    
    print('************ Files used: **************')
    # generate payoff table
    for i in range(1, num_row+1):
        for j in range(1, num_column+1):
            filename = f'./experiment_data/S{i+row_start}_S{j+column_start}{weighting_str}.csv'
            payoff_str = ''
            if os.path.isfile(filename):
                print('->', filename)
                df = load_csv(filename, 34)
                payoff =  calculate_payoffs(df)
                
                check_orig = f'./experiment_data/S{i+row_start}_S{j+column_start}_price_{weighting}.csv'
                check_bis = f'./experiment_data/S{i+row_start}_S{j+column_start}_price_bis_{weighting}.csv'

                if re.match(r'.*_price_\d.*', filename) and os.path.isfile(check_bis):
                    print('->', check_bis)
                    df_bis = load_csv(check_bis, 34)
                    bis_payoff = calculate_payoffs(df_bis)
                    avg_payoffs_ = cal_avg(payoff, bis_payoff)
                    payoff_str = f'[{avg_payoffs_[0]}, {avg_payoffs_[1]}]'
                elif re.match(r'.*_price_bis_\d.*', filename) and os.path.isfile(check_orig):
                    print('->', check_orig)
                    df_orig = load_csv(check_orig, 34)
                    orig_payoff = calculate_payoffs(df_orig)
                    avg_payoffs_ = cal_avg(payoff, orig_payoff)
                    payoff_str = f'[{avg_payoffs_[0]}, {avg_payoffs_[1]}]'
                else:
                    payoff_str = f'[{payoff[0]}, {payoff[1]}]'
                    
                payoffs[i-1][j-1] = payoff_str
                
    # mark dominant strategies in retailer 1's perspective
    for i in range(0, num_row):
        cur_row = payoffs[i]
        (dominant_index, dominant_str) = get_column_dominant(cur_row)
        payoffs[i][dominant_index] = dominant_str
    
    # mark dominant strategies in retailer 0's perspective
    for i in range(0, num_column):
        cur_column = [row[i] for row in payoffs]
        (dominant_index, dominant_str) = get_row_dominant(cur_column)
        payoffs[dominant_index][i] = dominant_str

    # convert to dataframe
    table = pd.DataFrame(payoffs, columns=columns)
    table = table.rename(index=index_dict)
    display(table)

# Payoff table of Price weight=2 => focus on NE (S4, S1)

In [124]:
general_index_for_S4_S1_single_column = {0: 'S1', 1: 'S2', 2: 'S3', 3: 'S4', 4: 'S5', 5: 'S6'}
general_index_for_S4_S1_single_row = {0:'S4'}
columns_for_S4_S1 = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']

In [125]:
construct_payoff_table_for_weightings(['S1'], 6, 1, '_price_', 2, general_index_for_S4_S1_single_column)

************ Files used: **************
-> ./experiment_data/S1_S1_price_2.csv
-> ./experiment_data/S2_S1_price_2.csv
-> ./experiment_data/S2_S1_price_bis_2.csv
-> ./experiment_data/S3_S1_price_2.csv
-> ./experiment_data/S4_S1_price_2.csv
-> ./experiment_data/S5_S1_price_2.csv
-> ./experiment_data/S6_S1_price_2.csv


,S1
S1,"[195018.75, 199574.35*]"
S2,"[253094.72*, 230575.55*]"
S3,"[234221.0, 253022.4*]"
S4,"[185731.0, 249841.35*]"
S5,"[231363.5, 224496.8*]"
S6,"[222494.0, 258300.6*]"


In [126]:
construct_payoff_table_for_weightings(columns_for_S4_S1, 1, 6, '_price_', 2, general_index_for_S4_S1_single_row, 3)

************ Files used: **************
-> ./experiment_data/S4_S1_price_2.csv
-> ./experiment_data/S4_S2_price_2.csv
-> ./experiment_data/S4_S3_price_2.csv
-> ./experiment_data/S4_S3_price_bis_2.csv
-> ./experiment_data/S4_S4_price_2.csv
-> ./experiment_data/S4_S5_price_2.csv
-> ./experiment_data/S4_S6_price_2.csv


,S1,S2,S3,S4,S5,S6
S4,"[185731.0*, 249841.35*]","[183350.75*, 216939.8]","[224823.38*, 239784.5]","[194708.0*, 226403.25]","[87107.0*, 102313.0]","[87651.75*, 60382.5]"


# Payoff table of Price weight=6 => focus on NE (S4, S1)

In [127]:
construct_payoff_table_for_weightings(['S1'], 6, 1, '_price_', 6, general_index_for_S4_S1)

************ Files used: **************
-> ./experiment_data/S1_S1_price_6.csv
-> ./experiment_data/S2_S1_price_6.csv
-> ./experiment_data/S2_S1_price_bis_6.csv
-> ./experiment_data/S3_S1_price_6.csv
-> ./experiment_data/S4_S1_price_6.csv
-> ./experiment_data/S5_S1_price_6.csv
-> ./experiment_data/S6_S1_price_6.csv


,S1
S1,"[210715.65, 132990.35*]"
S2,"[256517.47, 195554.15*]"
S3,"[242176.75, 197992.65*]"
S4,"[238284.25, 214720.25*]"
S5,"[237910.5, 199162.15*]"
S6,"[267071.0*, 218650.1*]"


In [128]:
construct_payoff_table_for_weightings(columns_for_S4_S1, 1, 6, '_price_', 6, general_index_for_S4_S1_single_row, 3)

************ Files used: **************
-> ./experiment_data/S4_S1_price_6.csv
-> ./experiment_data/S4_S2_price_6.csv
-> ./experiment_data/S4_S3_price_6.csv
-> ./experiment_data/S4_S3_price_bis_6.csv
-> ./experiment_data/S4_S4_price_6.csv
-> ./experiment_data/S4_S5_price_6.csv
-> ./experiment_data/S4_S6_price_6.csv


,S1,S2,S3,S4,S5,S6
S4,"[238284.25*, 214720.25*]","[125558.0*, 120248.4]","[167392.5*, 150284.25]","[164330.5*, 156036.0]","[155243.5*, 158404.5]","[105691.5*, 81768.5]"


# Payoff table of Price weight=10 => focus on NE (S4, S1)

In [129]:
construct_payoff_table_for_weightings(['S1'], 6, 1, '_price_', 10, general_index_for_S4_S1)

************ Files used: **************
-> ./experiment_data/S1_S1_price_10.csv
-> ./experiment_data/S2_S1_price_10.csv
-> ./experiment_data/S2_S1_price_bis_10.csv
-> ./experiment_data/S3_S1_price_10.csv
-> ./experiment_data/S4_S1_price_10.csv
-> ./experiment_data/S5_S1_price_10.csv
-> ./experiment_data/S6_S1_price_10.csv


,S1
S1,"[215655.3, 105533.1*]"
S2,"[229937.97, 162951.53*]"
S3,"[239466.75, 160783.75*]"
S4,"[248395.75*, 201163.6*]"
S5,"[224352.0, 177530.7*]"
S6,"[95830.5, 71869.5*]"


In [130]:
construct_payoff_table_for_weightings(columns_for_S4_S1, 1, 6, '_price_', 10, general_index_for_S4_S1_single_row, 3)

************ Files used: **************
-> ./experiment_data/S4_S1_price_10.csv
-> ./experiment_data/S4_S2_price_10.csv
-> ./experiment_data/S4_S3_price_10.csv
-> ./experiment_data/S4_S3_price_bis_10.csv
-> ./experiment_data/S4_S4_price_10.csv
-> ./experiment_data/S4_S5_price_10.csv
-> ./experiment_data/S4_S6_price_10.csv


,S1,S2,S3,S4,S5,S6
S4,"[248395.75*, 201163.6*]","[95675.25*, 91256.2]","[118569.12*, 103293.38]","[42682.5*, 40310.25]","[90856.0*, 76827.5]","[67475.25*, 58088.0]"


=========================================================

# Payoff table of Price weight=2 => focus on NE (S2, S3)

In [88]:
general_index_for_S2_S3_single_column = {0: 'S1', 1: 'S2', 2: 'S3', 3: 'S4', 4: 'S5', 5: 'S6'}
general_index_for_S2_S3_single_row = {0:'S2'}
columns_for_S2_S3 = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']

In [131]:
construct_payoff_table_for_weightings(['S3'], 6, 1, '_price_bis_', 2, general_index_for_S2_S3_single_column, 0, 2)

************ Files used: **************
-> ./experiment_data/S1_S3_price_bis_2.csv
-> ./experiment_data/S2_S3_price_bis_2.csv
-> ./experiment_data/S3_S3_price_bis_2.csv
-> ./experiment_data/S4_S3_price_bis_2.csv
-> ./experiment_data/S4_S3_price_2.csv
-> ./experiment_data/S5_S3_price_bis_2.csv
-> ./experiment_data/S6_S3_price_bis_2.csv


,S3
S1,"[136026.35, 131747.5*]"
S2,"[194366.5, 142389.5*]"
S3,"[180784.25, 149406.0*]"
S4,"[224823.38*, 239784.5*]"
S5,"[142405.5, 119375.25*]"
S6,"[144640.0, 111529.5*]"


In [132]:
construct_payoff_table_for_weightings(columns_for_S2_S3, 1, 6, '_price_bis_', 2, general_index_for_S2_S3_single_row, 1, 0)


************ Files used: **************
-> ./experiment_data/S2_S1_price_bis_2.csv
-> ./experiment_data/S2_S1_price_2.csv
-> ./experiment_data/S2_S2_price_bis_2.csv
-> ./experiment_data/S2_S3_price_bis_2.csv
-> ./experiment_data/S2_S4_price_bis_2.csv
-> ./experiment_data/S2_S5_price_bis_2.csv
-> ./experiment_data/S2_S6_price_bis_2.csv


,S1,S2,S3,S4,S5,S6
S2,"[253094.72*, 230575.55*]","[196914.05*, 216158.3]","[194366.5*, 142389.5]","[108463.7*, 138837.75]","[114539.75*, 113252.0]","[103210.05*, 99662.5]"


# Payoff table of Price weight=6 => focus on NE (S2, S3)

In [133]:
construct_payoff_table_for_weightings(['S3'], 6, 1, '_price_bis_', 6, general_index_for_S2_S3_single_column, 0, 2)

************ Files used: **************
-> ./experiment_data/S1_S3_price_bis_6.csv
-> ./experiment_data/S2_S3_price_bis_6.csv
-> ./experiment_data/S3_S3_price_bis_6.csv
-> ./experiment_data/S4_S3_price_bis_6.csv
-> ./experiment_data/S4_S3_price_6.csv
-> ./experiment_data/S5_S3_price_bis_6.csv
-> ./experiment_data/S6_S3_price_bis_6.csv


,S3
S1,"[141046.7, 117104.25*]"
S2,"[148739.3, 116827.75*]"
S3,"[114730.5, 91419.25*]"
S4,"[167392.5*, 150284.25*]"
S5,"[124111.5, 115753.75*]"
S6,"[118846.5, 114537.5*]"


In [134]:
construct_payoff_table_for_weightings(columns_for_S2_S3, 1, 6, '_price_bis_', 6, general_index_for_S2_S3_single_row, 1, 0)


************ Files used: **************
-> ./experiment_data/S2_S1_price_bis_6.csv
-> ./experiment_data/S2_S1_price_6.csv
-> ./experiment_data/S2_S2_price_bis_6.csv
-> ./experiment_data/S2_S3_price_bis_6.csv
-> ./experiment_data/S2_S4_price_bis_6.csv
-> ./experiment_data/S2_S5_price_bis_6.csv
-> ./experiment_data/S2_S6_price_bis_6.csv


,S1,S2,S3,S4,S5,S6
S2,"[256517.47*, 195554.15*]","[150891.65*, 118761.5]","[148739.3*, 116827.75]","[75196.85*, 70558.0]","[96448.2*, 78840.0]","[67154.5*, 55146.5]"


# Payoff table of Price weight=10 => focus on NE (S2, S3)

In [135]:
construct_payoff_table_for_weightings(['S3'], 6, 1, '_price_bis_', 10, general_index_for_S2_S3_single_column, 0, 2)

************ Files used: **************
-> ./experiment_data/S1_S3_price_bis_10.csv
-> ./experiment_data/S2_S3_price_bis_10.csv
-> ./experiment_data/S3_S3_price_bis_10.csv
-> ./experiment_data/S4_S3_price_bis_10.csv
-> ./experiment_data/S4_S3_price_10.csv
-> ./experiment_data/S5_S3_price_bis_10.csv
-> ./experiment_data/S6_S3_price_bis_10.csv


,S3
S1,"[145578.7*, 111322.0*]"
S2,"[112578.15, 88997.75*]"
S3,"[92956.0, 95276.75*]"
S4,"[118569.12, 103293.38*]"
S5,"[107967.5, 95881.25*]"
S6,"[109644.0, 91476.0*]"


In [136]:
construct_payoff_table_for_weightings(columns_for_S2_S3, 1, 6, '_price_bis_', 10, general_index_for_S2_S3_single_row, 1, 0)


************ Files used: **************
-> ./experiment_data/S2_S1_price_bis_10.csv
-> ./experiment_data/S2_S1_price_10.csv
-> ./experiment_data/S2_S2_price_bis_10.csv
-> ./experiment_data/S2_S3_price_bis_10.csv
-> ./experiment_data/S2_S4_price_bis_10.csv
-> ./experiment_data/S2_S5_price_bis_10.csv
-> ./experiment_data/S2_S6_price_bis_10.csv


,S1,S2,S3,S4,S5,S6
S2,"[229937.97*, 162951.53*]","[112841.2*, 78321.2]","[112578.15*, 88997.75]","[46013.4*, 50646.25]","[98552.75*, 71599.5]","[74543.35*, 53361.0]"
